<a href="https://colab.research.google.com/github/papaymaguire/ece219-project2/blob/main/project2/notebooks/part03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>